In [1]:
#Packages
from pathlib import Path
import exifread
import matplotlib.image
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray
import astropy.units as u
from sunpy.time import parse_time
import sunpy.coordinates
import sunpy.coordinates.sun
from astropy.constants import R_earth
from astropy.coordinates import CartesianRepresentation, EarthLocation, SkyCoord
import math
from scipy import ndimage
from skimage.transform import hough_circle, hough_circle_peaks
from matplotlib.patches import Circle
from astropy.coordinates import CartesianRepresentation, EarthLocation, SkyCoord
from sunpy.map.header_helper import make_fitswcs_header
from skimage.feature import peak_local_max
from astropy.time import Time
from sunpy.coordinates import frames
from sunpy.coordinates import sun
import os
import random
import rawpy
import imageio
import csv
from astropy.io import fits
from scipy.ndimage import rotate

F:\rotation_eclipse\jupyter\py310_work_version\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

#Not used in the final version of this project

def readCSV(user):
    '''
        Read the data.csv file and get the lat, lon, and time stored there
        user<string>: name of the user ID associated with the upload
    '''
    with open('data.csv', mode='r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if row[0] == user:
                return float(row[1]), float(row[2]), parse_time(row[3]), parse_time(row[4])
    return False

In [3]:
def readCSVwFile(file_name):
    '''
        Read the data.csv file and get the lat, lon, and time stored there
        file_name<string>: name of the file
    '''
    with open('test_exif/data_w_times.csv', mode='r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if row[1] == file_name:
                return float(row[2]), float(row[3]), parse_time(row[4]), parse_time(row[5])
    return False

In [4]:
def reset():
    '''
        Wipe file and reset the headers
    '''
    with open('out_new.csv', mode='w') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Folder', 'FileName', 'Angle'])
        
def addToCSV(user_id, file_name, angle):
    '''
        Add row of information to the CSV file
    '''
    with open('out_new.csv', mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow([user_id, file_name, angle])

In [5]:
all_users = os.listdir('../24_images')
all_users.remove('.ipynb_checkpoints') #remove checkpoint from the list
raw_types = ['CR3', 'cr3', 'RAF', 'ARW', 'nef']
#3149 total files

for user in all_users:
    all_images = os.listdir('../24_images/' + user)
    for file_name in all_images:
        try:
            SOLAR_ECLIPSE_IMAGE = '../24_images/' + user + '/' + file_name
            lat, lon, time, utc = readCSVwFile(file_name)
            time = utc
            print(f'lat:{lat}, lon:{lon}, time:{time}, utc:{utc} file:{SOLAR_ECLIPSE_IMAGE}')

            skip = False
            #Don't retread on information we already have
            with open('out_new.csv', mode='r') as file:
                csv_reader = csv.reader(file)
                for row in csv_reader:
                    if row[1] == file_name:
                        skip = True
                        print(f'{file_name} is already in the csv file')
                        break
                if skip:
                    continue
            
                
                
            # if file_name.split('.')[-1].lower() != 'cr2':
            #     with open('out.csv', mode='r') as file:
            #         csv_reader = csv.reader(file)
            #         for row in csv_reader:
            #             print(row, len(row))
            #             if len(row) < 3:
            #                 continue
            #             if row[0] == user:
            #                 skip = True
            #                 print(f'Not a cr2 file, adding {file_name}')
            #                 addToCSV(user, file_name, row[2])
            #                 break
            #         if skip:
            #             continue



                    
            
            if SOLAR_ECLIPSE_IMAGE.split('.')[-1] in raw_types:
                with rawpy.imread(SOLAR_ECLIPSE_IMAGE) as raw:
                    rgb_image = raw.postprocess()
                im_rgb = np.flipud(rgb_image)
                im = np.dot(im_rgb[...,:3], [0.2989, 0.5870, 0.1140]) 
            elif SOLAR_ECLIPSE_IMAGE.split('.')[-1] == 'fits':
                with fits.open(SOLAR_ECLIPSE_IMAGE) as hdul:
                    im = hdul[0].data
            else:
                im_rgb = np.flipud(matplotlib.image.imread(SOLAR_ECLIPSE_IMAGE))
                im = rgb2gray(im_rgb)  
    
            #Find the moon
            blur_im = ndimage.gaussian_filter(im, 8)
            mask = blur_im > blur_im.mean() * 3
            
            
            label_im, nb_labels = ndimage.label(mask)
            slice_y, slice_x = ndimage.find_objects(label_im == 1)[0]
            roi = blur_im[slice_y, slice_x]
            
            sx = ndimage.sobel(roi, axis=1, mode="constant")
            sy = ndimage.sobel(roi, axis=0, mode="constant")
            sob = np.hypot(sx, sy)
            
            hough_radii = np.arange(np.floor(np.mean(sob.shape) / 4), np.ceil(np.mean(sob.shape) / 2), 10)
            hough_res = hough_circle(sob > (sob.mean() * 5), hough_radii)
            
            # Select the most prominent circle
            accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)
            
            #pixel values for the center of the moon
            im_cx = (cx[0] + slice_x.start) * u.pix
            im_cy = (cy[0] + slice_y.start) * u.pix
            im_radius = radii[0] * u.pix
    
            #sunpy map
            loc = EarthLocation(lat=lat, lon=lon, height=0)
            observer = loc.get_itrs(time)
    
            moon = SkyCoord(sunpy.coordinates.get_body_heliographic_stonyhurst("moon", time, observer=observer))
            R_moon = 0.2725076 * R_earth  # IAU mean radius
            dist_moon = SkyCoord(observer).separation_3d(moon)
            moon_obs = np.arcsin(R_moon / dist_moon).to("arcsec")
            plate_scale = moon_obs / im_radius
            solar_rotation_angle = sunpy.coordinates.sun.orientation(loc, time)
    
            frame = sunpy.coordinates.Helioprojective(observer=observer, obstime=time)
            moon_hpc = moon.transform_to(frame)
            
            header = make_fitswcs_header(
                im,
                moon_hpc,
                reference_pixel=u.Quantity([im_cx, im_cy]),
                scale=u.Quantity([plate_scale, plate_scale]),
                rotation_angle=solar_rotation_angle,
            )
            eclipse_map = sunpy.map.Map(im, header)
    
    
            #Find solar north
            obs_location = SkyCoord(lon * u.deg, lat * u.deg, frame=frames.HeliographicStonyhurst, obstime=time)
            # Calculate the position of solar north
            solar_north_angle = sun.P(time)
            solar_north_coord = SkyCoord(0 * u.arcsec, 0 * u.arcsec, frame=eclipse_map.coordinate_frame)
            solar_north_coord = solar_north_coord.transform_to(frames.Helioprojective(observer=obs_location))
            solar_north_coord = solar_north_coord.directional_offset_by(solar_north_angle, 1000 * u.arcsec)
            
            solar_north_pixel = eclipse_map.world_to_pixel(solar_north_coord)
            
            solar_north_x = solar_north_pixel.x.to_value()
            solar_north_y = solar_north_pixel.y.to_value()
    
    
            # Ensure the quantities are converted to dimensionless scalars
            solar_north_x_value = solar_north_pixel.x.to_value(u.pix)
            solar_north_y_value = solar_north_pixel.y.to_value(u.pix)
            im_cx_value = im_cx.to_value(u.pix)
            im_cy_value = im_cy.to_value(u.pix)
            
            # Define the points using the dimensionless values
            A = np.array([im_cx_value, im_cy_value])
            B = np.array([solar_north_x_value, solar_north_y_value])
            C = np.array([im_cx_value, solar_north_y_value])
            
            # Calculate the vectors
            AB = B - A
            AC = C - A
            
            # Calculate dot product
            dot_product = np.dot(AB, AC)
            
            # Calculate magnitudes
            magnitude_AB = np.linalg.norm(AB)
            magnitude_AC = np.linalg.norm(AC)
            
            # Calculate cosine of the angle
            cos_angle = dot_product / (magnitude_AB * magnitude_AC)
            
            # Calculate the angle in radians
            angle_radians = np.arccos(cos_angle)
            
            # Convert the angle to degrees
            angle_degrees = np.degrees(angle_radians)
            
            print(f"The angle between the points is {angle_degrees:.2f} degrees")
    
            rotation_angle = angle_degrees
                        
            rotated_image = rotate(im, rotation_angle, reshape=True)
    
            addToCSV(user, file_name, rotation_angle)

            # break
        except Exception as e: 
            print(f'!!!oops, an issue with {user}/{file_name}: {e}')

lat:29.22697, lon:-100.89242, time:2024-04-08T17:28:07.000, utc:2024-04-08T17:28:07.000 file:../24_images/R_32LU6pKyipqNvLD/IMG_4772.CR2
IMG_4772.CR2 is already in the csv file
lat:29.22697, lon:-100.89242, time:2024-04-08T17:28:08.000, utc:2024-04-08T17:28:08.000 file:../24_images/R_32LU6pKyipqNvLD/IMG_4773.CR2
IMG_4773.CR2 is already in the csv file
lat:29.22697, lon:-100.89242, time:2024-04-08T17:28:09.000, utc:2024-04-08T17:28:09.000 file:../24_images/R_32LU6pKyipqNvLD/IMG_4774.CR2
IMG_4774.CR2 is already in the csv file
lat:29.22697, lon:-100.89242, time:2024-04-08T17:28:10.000, utc:2024-04-08T17:28:10.000 file:../24_images/R_32LU6pKyipqNvLD/IMG_4775.CR2
IMG_4775.CR2 is already in the csv file
lat:29.22697, lon:-100.89242, time:2024-04-08T17:28:13.000, utc:2024-04-08T17:28:13.000 file:../24_images/R_32LU6pKyipqNvLD/IMG_4776.CR2
!!!oops, an issue with R_32LU6pKyipqNvLD/IMG_4776.CR2: index 0 is out of bounds for axis 0 with size 0
lat:29.22697, lon:-100.89242, time:2024-04-08T17:28: